Import Packages

In [1]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiLineString, LineString
from shapely.ops import nearest_points
import math
import numpy as np

Load Data from our API Calls
- the trunkroute needs to be updated to include direction == 2 (krystal)

In [2]:
trunkroute = pd.read_csv("Bus_RoutesStopsServices/trunkroute.csv")
transport_node_bus_202407 = pd.read_csv("Passenger_Volume_By_Bus_Stop/transport_node_bus_202407.csv")
transport_node_bus_202408 = pd.read_csv("Passenger_Volume_By_Bus_Stop/transport_node_bus_202408.csv")
transport_node_bus_202409 = pd.read_csv("Passenger_Volume_By_Bus_Stop/transport_node_bus_202409.csv")

MRT line mapping
- mrt_lines mapping
- function to find MRT line based on bus stop description

In [3]:
#mapping of stations to MRT lines
mrt_lines = {
    'Thomson-East Coast Line': ['WOODLANDS NORTH MRT STATION','WOODLANDS MRT STATION','WOODLANDS SOUTH MRT STATION', 'SPRINGLEAF MRT STATION','LENTOR MRT STATION','MAYFLOWER MRT STATION','BRIGHT HILL MRT STATION',
                                'UPPER THOMSON MRT STATION','CALDECOTT MRT STATION','STEVENS MRT STATION','NAPIER MRT STATION','ORCHARD BOULEVARD MRT STATION','ORCHARD MRT STATION','GREAT WORLD MRT STATION',
                                'HAVELOCK MRT STATION','OUTRAM PARK MRT STATION','MAXWELL MRT STATION','SHENTON WAY MRT STATION','MARINA BAY MRT STATION','GARDENS BY THE BAY MRT STATION','TANJONG RHU MRT STATION',
                                'KATONG PARK MRT STATION','TANJONG KATONG MRT STATION','MARINE PARADE MRT STATION','MARINE TERRACE MRT STATION','SIGLAP MRT STATION','BAYSHORE MRT STATION'],
    'North-South Line': ['JURONG EAST MRT STATION','BUKIT BATOK MRT STATION','BUKIT GOMBAK MRT STATION','CHOA CHU KANG MRT STATION','YEW TEE MRT STATION','KRANJI MRT STATION','MARSILING MRT STATION','WOODLANDS MRT STATION',
                         'ADMIRALTY MRT STATION','SEMBAWANG MRT STATION','CANBERRRA MRT STATION','YISHUN MRT STATION','KHATIB MRT STATION','YIO CHU KANG MRT STATION','ANG MO KIO MRT STATION','BISHAN MRT STATION',
                         'BRADDELL MRT STATION','TOA PAYOH MRT STATION','NOVENA MRT STATION','NEWTON MRT STATION','ORCHARD MRT STATION','SOMERSET MRT STATION','DHOBY GHAUT MRT STATION','CITY HALL MRT STATION',
                         'RAFFLES PLACE MRT STATION','MARINA BAY MRT STATION','MARINA SOUTH PIER MRT STATION'],
    'Circle Line': ['DHOBY GHAUT MRT STATION', 'BRAS BASAH MRT STATION', 'ESPLANADE MRT STATION', 'PROMENADE MRT STATION','NICOLL HIGHWAY MRT STATION','STADIUM MRT STATION',
                    'MOUNTBATTEN MRT STATION','DAKOTA MRT STATION','PAYA LEBAR MRT STATION','MACPHERSON MRT STATION','TAI SENG MRT STATION','BARTLEY MRT STATION','SERANGOON MRT STATION','LORONG CHUAN MRT STATION',
                    'BISHAN MRT STATION','MARYMOUNT MRT STATION','CALDECOTT MRT STATION','BOTANIC GARDENS MRT STATION','FARRER ROAD MRT STATION','HOLLAND VILLAGE MRT STATION','BUONA VISTA MRT STATION','ONE-NORTH MRT STATION',
                    'KENT RIDGE MRT STATION', 'HAW PAW VILLA MRT STATION', 'PASIR PANJANG MRT STATION', 'LABRADOR PARK MRT STATION', 'TELOK BLANGAH MRT STATION','HABOURFRONT MRT STATION'],
    'Circle Line.': ['PROMENADE MRT STATION','BAYFRONT MRT STATION', 'MARINA BAY MRT STATION'],
    'North-East Line': ['HARBOURFRONT MRT STATION','OUTRAM PARK MRT STATION','CHINATOWN MRT STATION','CLARKE QUAY MRT STATION','DHOBY GHAUT MRT STATION','LITTLE INDIA MRT STATION','FARRER PARK MRT STATION','BOON KENG MRT STATION',
                        'POTONG PASIR MRT STATION','WOODLEIGH MRT STATION','SERANGOON MRT STATION','KOVAN MRT STATION','HOUGANG MRT STATION','BUANGKOK MRT STATION','SENGKANG MRT STATION','PUNGGOL MRT STATION'],
    'Downtown Line': ['BUKIT PANJANG MRT STATION', 'CASHEW MRT STATION', 'HILLVIEW MRT STATION', 'HUME MRT STATION', 'BEAUTY WORLD MRT STATION', 'KING ALBERT PARK MRT STATION', 'SIXTH AVENUE','TAN KAH KEE MRT STATION',
                      'BOTANIC GARDENS MRT STATION', 'STEVENS MRT STATION', 'NEWTON MRT STATION', 'LITTLE INDIA MRT STATION', 'ROCHOR MRT STATION', 'BUGIS MRT STATION', 'PROMENADE MRT STATION', 'BAYFRONT MRT STATION',
                      'DOWNTOWN MRT STATION', 'TELOK AYER MRT STATION', 'CHINATOWN MRT STATION', 'FORT CANNING MRT STATION', ' BENCOOLEN MRT STATION', 'JALAN BESAR MRT STATION', 'BENDEMEER MRT STATION', 'GEYLANG  BAHRU MRT STATION',
                      'MATTAR MRT STATION', 'MACPHERSON MRT STATION', 'UBI MRT STATION', 'KAKI BUKIT MRT STATION', ' BEDOK NORTH MRT STATION', 'BEDOK RESERVOIR MRT STATION', 'TAMPINES WEST MRT STATION',
                      'TAMPINES MRT STATION', 'TAMPINES EAST MRT STATION', 'UPPER CHANGI MRT STATION', 'EXPO MRT STATION'],
    'East-West Line': ['PASIR RIS MRT STATION', 'TAMPINES MRT STATION', 'SIMEI MRT STATION', 'TANAH MERAH MRT STATION','BEDOK MRT STATION', 'KEMBANGAN MRT STATION', 'EUNOS MRT STATION', 'PAYA LEBAR MRT STATION',
                       'ALJUNIED MRT STATION', 'KALLANG MRT STATION', ' LAVENDER MRT STATION','BUGIS MRT STATION', 'CITY HALL MRT STATION', 'RAFFLES PLACE MRT STATION','TANJONG PAGAR MRT STATION', 'OUTRAM PARK MRT STATION',
                       'TIONG BAHRU MRT STATION', 'REDHILL MRT STATION', 'QUEENSTOWN MRT STATION', 'COMMONWEALTH MRT STATION','BUONA VISTA MRT STATION', 'DOVER MRT STATION', 'CLEMENTI MRT STATION', 'JURONG EAST MRT STATION',
                       'CHINESE GARDEN MRT STATION', 'LAKESIDE MRT STATION', 'BOON LAY MRT STATION', 'PIONEER MRT STATION','JOO KON MRT STATION',' GUL CIRCLE MRT STATION', 'TUAS CRESCENT MRT STATION', 
                       'TUAS WEST ROAD MRT STATION', 'TUAS LINK MRT STATION'],
    'East-West Line.':['TANAH MERAH MRT STATION','EXPO MRT STATION', 'CHANGI AIRPORT MRT STATION']
}

def find_mrt_line(description, mrt_lines):

    matching_lines = []

    for line, stations in mrt_lines.items():
        for station in stations:
            station_name = station.replace('MRT STATION', '').strip().lower() 
            if station_name in description.lower(): 
                matching_lines.append(line)
    return matching_lines if matching_lines else None

Bus Stop Passenger Volume Data

- PT_CODE: bus stop code
- average_passenger_volume: sum of weekday & weekend tap in & tap out for each bus service, averaged across 3 months



In [9]:
#for each bus stop - get sum of tap in + tap out
jul_busstop_vol = transport_node_bus_202407.groupby(['PT_TYPE', 'PT_CODE']).agg(
    july_tap_in = ('TOTAL_TAP_IN_VOLUME', 'sum'),
    july_tap_out = ('TOTAL_TAP_OUT_VOLUME', 'sum')
).reset_index()

aug_busstop_vol = transport_node_bus_202408.groupby(['PT_TYPE', 'PT_CODE']).agg(
    august_tap_in = ('TOTAL_TAP_IN_VOLUME', 'sum'),
    august_tap_out = ('TOTAL_TAP_OUT_VOLUME', 'sum')
).reset_index()

sep_busstop_vol = transport_node_bus_202409.groupby(['PT_TYPE', 'PT_CODE']).agg(
    september_tap_in = ('TOTAL_TAP_IN_VOLUME', 'sum'),
    september_tap_out = ('TOTAL_TAP_OUT_VOLUME', 'sum')
).reset_index()

#for each bus stop - get passenger volume (tap in + tap out)
jul_busstop_vol['july_volume'] = jul_busstop_vol['july_tap_in'] + jul_busstop_vol['july_tap_out']
jul_busstop_vol = jul_busstop_vol.drop(columns = ['july_tap_in', 'july_tap_out'])

aug_busstop_vol['august_volume'] = aug_busstop_vol['august_tap_in'] + aug_busstop_vol['august_tap_out']
aug_busstop_vol = aug_busstop_vol.drop(columns = ['august_tap_in', 'august_tap_out'])

sep_busstop_vol['september_volume'] = sep_busstop_vol['september_tap_in'] + sep_busstop_vol['september_tap_out']
sep_busstop_vol = sep_busstop_vol.drop(columns = ['september_tap_in', 'september_tap_out'])

#merge to get all months volumes
busstop_volumes = pd.merge(jul_busstop_vol, aug_busstop_vol, on = ['PT_TYPE', 'PT_CODE'], how = 'outer')
busstop_volumes = pd.merge(busstop_volumes, sep_busstop_vol, on = ['PT_TYPE', 'PT_CODE'], how = 'outer')

#calc average passenger volume across the 3 months
busstop_volumes['average_passenger_volume'] = busstop_volumes[['july_volume', 'august_volume', 'september_volume']].mean(axis = 1)
busstop_volumes = busstop_volumes.drop(columns=['july_volume', 'august_volume', 'september_volume', 'PT_TYPE'])

Combined Bus Data
- to be used for this remaining file

In [10]:
#merge trunkroute and busstop_volumes on BusStopCode = PT_CODE
combined_bus_data = pd.merge(trunkroute, busstop_volumes, left_on = 'BusStopCode', right_on = 'PT_CODE', how = 'inner')

#add column: if the bus stop is MRT bus stop or not
combined_bus_data['MRTBusStop'] = combined_bus_data['Description'].apply(lambda x: 1 if 'Stn' in x or 'Int' in x else 0)

#add column: to find MRTline(s) of MRT bus stops
combined_bus_data['MRTLine'] = combined_bus_data.apply(
    lambda row: find_mrt_line(row['Description'], mrt_lines) if row['MRTBusStop'] == 1 else None, axis=1
)

#select only the relevant columns
combined_bus_data = combined_bus_data[['ServiceNo', 'Direction', 'StopSequence', 'BusStopCode', 'MRTBusStop', 'MRTLine',
                                       'average_passenger_volume', 'Latitude', 'Longitude']]

print(combined_bus_data.head())


  ServiceNo  Direction  StopSequence  BusStopCode  MRTBusStop  \
0        10          1             1        75009           1   
1        10          1             2        76059           0   
2        10          1             3        76069           0   
3        10          1             4        96289           0   
4        10          1             5        96109           0   

                           MRTLine  average_passenger_volume  Latitude  \
0  [Downtown Line, East-West Line]              2.562661e+06  1.354076   
1                             None              1.545133e+05  1.352962   
2                             None              1.494460e+05  1.348753   
3                             None              1.428963e+05  1.340055   
4                             None              5.027333e+04  1.337371   

    Longitude  
0  103.943391  
1  103.941652  
2  103.942086  
3  103.948381  
4  103.950673  


Top 5 bus services from parallelism_df 
- hardcoded to test, but to take data from parallelscore.ipynb




In [37]:
parallelism_df = {
    'ServiceNo': ['63', '961M', '145', '36', '63A', '6', '127A', '973A', '68A', '88B'],
    'Direction': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'ParallelismScore': [2.254552, 1.964276, 1.927232, 1.901650, 1.872716, 0.004168, 0.003957, 0.003083, 0.003008, 0.002643]
}

parallelism_df = pd.DataFrame(parallelism_df)
top_5_buses = parallelism_df.sort_values(by = 'ParallelismScore', ascending = False).head(5)
top_routes = combined_bus_data[combined_bus_data['ServiceNo'].isin(top_5_buses['ServiceNo'])].copy()

Input inside and outside threshold (can be changed)
- currently set as 0.5 (average)
- increasing this quantile value will increase the threshold value

In [38]:
inside_quantile = 0.5
outside_quantile = 0.5

Function to get actual threshold values
1) all 'in between' MRT bus stops threshold: distinct bus stops that are between MRT bus stops, in singapore
2) all bus stops in each route threshold: distinct bus stops for the bus service, in that bus route

In [65]:
#all 'in between' MRT bus stops threshold 

#for each bus service,
# # that have at least 2 bus stops, store the busstop codes that are in between the first and last MRT bus stops
# if its inside the list, dont include it (to get distinct)

def create_inner_threshold(combined_bus_data, inside_quantile):

    in_between_volumes = []
    grouped_routes = combined_bus_data.groupby(['ServiceNo', 'Direction'])
    processed_stops = set()

    #for each route
    for (service_no, direction), route in grouped_routes:

        route = route.sort_values(by='StopSequence')
        mrt_busstops = route[route['MRTBusStop'] == 1]

        if len(mrt_busstops) >= 2:

            first_mrt_stop_seq = mrt_busstops['StopSequence'].min()
            last_mrt_stop_seq = mrt_busstops['StopSequence'].max()

            #non-MRT bus stops in between the first and last MRT bus stop
            in_between_stops = route[(route['StopSequence'] > first_mrt_stop_seq) &
                                     (route['StopSequence'] < last_mrt_stop_seq) &
                                     (route['MRTBusStop'] == 0)]

            in_between_stops = in_between_stops[~in_between_stops['BusStopCode'].isin(processed_stops)]

            #add distinct busstop volumes
            if not in_between_stops.empty:
                for _, stop in in_between_stops.iterrows():
                    if pd.notna(stop['average_passenger_volume']):
                        in_between_volumes.append(stop['average_passenger_volume'])

            #update processed stops
            processed_stops.update(in_between_stops['BusStopCode'])

    #get quantile value
    if in_between_volumes:
        in_between_volumes_series = pd.Series(in_between_volumes)
        inside_threshold_value = in_between_volumes_series.quantile(inside_quantile)
    else:
        inside_threshold_value = None 

    return inside_threshold_value

In [60]:
#route = combined_bus_data[combined_bus_data['ServiceNo'] == 'bus number that we want']
def create_outer_threshold(route, outside_quantile):

    outside_volumes = []

    route = route.sort_values(by = 'StopSequence')
    mrt_busstops = route[route['MRTBusStop'] == 1]

    #check if got at least 2 mrt stops
    if len(mrt_busstops) >= 2:

        #get stop sequences of the outside busstops
        first_mrt_stop_seq = mrt_busstops['StopSequence'].min()
        last_mrt_stop_seq = mrt_busstops['StopSequence'].max()
        outside_stops = route[(route['StopSequence'] < first_mrt_stop_seq) | 
                                  (route['StopSequence'] > last_mrt_stop_seq)]

        if not outside_stops.empty:
            outside_volumes.extend(outside_stops['average_passenger_volume'].dropna().tolist())

    #get quantile value
    if outside_volumes:
        outside_volumes_series = pd.Series(outside_volumes)
        outside_threshold_value = outside_volumes_series.quantile(outside_quantile)
    else:
        outside_threshold_value = None  

    return outside_threshold_value


In [63]:
#inside threshold value
inside_threshold_value = create_inner_threshold(combined_bus_data, inside_quantile)

#outside threshold value for bus service 95 (js to test)
route = combined_bus_data[combined_bus_data['ServiceNo'] == '95']
outside_threshold_value = create_outer_threshold(route, outside_quantile)

print(f"Inner threshold value: {inside_threshold_value}")
print(f"Outer threshold value: {outside_threshold_value}")

Inner threshold value: 24812.666666666668
Outer threshold value: 10333.166666666668


Classification function to remove/keep bus stops (need change)


In [32]:
def process_bus_routes(combined_bus_data, top_buses, low_volume):

    #filter for top routes from the bus data and grp by bus
    top_routes = combined_bus_data[combined_bus_data['ServiceNo'].isin(top_buses['ServiceNo'])].copy()
    grouped_routes = top_routes.groupby(['ServiceNo', 'Direction'])

    #initialisation
    top_routes.loc[:, 'outcome'] = 'keep'
    final_routes = []

    for (service_no, direction), group in grouped_routes:

        group = group.sort_values(by = 'StopSequence')
        mrt_busstops = group[group['MRTBusStop'] == 1]
        
        #if route have < 2 MRT stops, keep the route
        if len(mrt_busstops) < 2:
            group.loc[:, 'outcome'] = 'keep'

        else:

            stop_sequences = mrt_busstops['StopSequence'].tolist()
            
            #iterate over each pairs of MRT stops
            for i in range(len(stop_sequences) - 1):

                #get seq number of both mrt bus stops
                first_mrt_busstop_sequence_no = stop_sequences[i]
                second_mrt_busstop_sequence_no = stop_sequences[i + 1]
                
                #calculate the average passenger volume for ALL stops in between the 1st and 2nd MRT bus stop
                in_between_stops = group[(group['StopSequence'] > first_mrt_busstop_sequence_no) & (group['StopSequence'] < second_mrt_busstop_sequence_no)]
                avg_in_between_volume = in_between_stops['average_passenger_volume'].mean()
                
                #if average vol is low, check outside range
                if avg_in_between_volume < low_volume:

                    #get seq number of before and after the 2 MRT bus stops
                    before_mrt = group[group['StopSequence'] < first_mrt_busstop_sequence_no]['average_passenger_volume'].mean()
                    after_mrt = group[group['StopSequence'] > second_mrt_busstop_sequence_no]['average_passenger_volume'].mean()
                    
                    #if outside range is low, remove all stops
                    if before_mrt < low_volume and after_mrt < low_volume:
                        group.loc[:, 'outcome'] = 'remove'

                    else:
                        #check if the MRT lines between the 1st and 2nd MRT busstops are matching
                        mrt_line_1 = mrt_busstops.iloc[i]['MRTLine'] if mrt_busstops.iloc[i]['MRTLine'] else []
                        mrt_line_2 = mrt_busstops.iloc[i + 1]['MRTLine'] if mrt_busstops.iloc[i + 1]['MRTLine'] else []
                        
                        mrt_lines_between = set(mrt_line_1).intersection(set(mrt_line_2))

                        #if MRT lines are matching, remove stops
                        if mrt_lines_between:
                            group.loc[(group['StopSequence'] > first_mrt_busstop_sequence_no) & (group['StopSequence'] < second_mrt_busstop_sequence_no), 'outcome'] = 'remove'
                        
                        #else, keep it
                        else:
                            group.loc[(group['StopSequence'] > first_mrt_busstop_sequence_no) & (group['StopSequence'] < second_mrt_busstop_sequence_no), 'outcome'] = 'keep'
                else:
                    #keep the in-between stops if volume is high
                    group.loc[(group['StopSequence'] > first_mrt_busstop_sequence_no) & (group['StopSequence'] < second_mrt_busstop_sequence_no), 'outcome'] = 'keep'

        final_routes.append(group)
    final_result = pd.concat(final_routes)

    return final_result



In [34]:

top_bus_routes = process_bus_routes(combined_bus_data, top_5_buses, inside_threshold_value)

output_csv_path = "top_bus_routes.csv"
top_bus_routes.to_csv(output_csv_path, index = False) 
print(f"Top bus routes saved to {output_csv_path}")

      ServiceNo  Direction  StopSequence  BusStopCode  MRTBusStop  \
2470        145          1             1        52009           1   
2471        145          1             2        52079           0   
2472        145          1             3        50161           0   
2473        145          1             4        50171           0   
2474        145          1             5        50191           0   
...         ...        ...           ...          ...         ...   
13283      961M          1            85         7371           0   
13284      961M          1            86        60011           0   
13285      961M          1            87        60059           0   
13286      961M          1            88        80109           1   
13287      961M          1            89        80009           0   

                  MRTLine  average_passenger_volume  Latitude   Longitude  \
2470   [North-South Line]              1.812367e+06  1.332021  103.846928   
2471             